In [37]:
import os
import json
import pandas as pd
import traceback

In [42]:
from langchain_openai import ChatOpenAI

In [43]:
from dotenv import load_dotenv

load_dotenv()

True

In [44]:
KEY=os.getenv("OPENAI_API_KEY") #把OPENAI_API_KEY 保存在环境中，方便后续调用。

In [49]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [252]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_community.callbacks import get_openai_callback
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnableParallel,RunnablePassthrough
import PyPDF2

In [253]:


RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer"
    },
}

In [254]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON  below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [255]:
quiz_generation_prompt= PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE,
)

In [256]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz',verbose=True)
# quiz_chain = quiz_generation_prompt | llm 

In [257]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for the complexity. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [258]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=['quiz','subject'],
    template=TEMPLATE2,
)

In [259]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key='review',verbose=True)

# review_chain= {'quiz': quiz_chain , 'subject': RunnablePassthrough()} | quiz_evaluation_prompt | llm | StrOutputParser()

In [260]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],output_variables=["quiz","review"],verbose=True)

In [261]:
file_path=r"/Users/jok/Desktop/00盒子/01项目/03MCQGenerator/mcqgen/data.txt"

In [262]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [263]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [264]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [266]:
#https://python.langchain.com/v0.1/docs/modules/model_io/llms/token_usage_tracking/
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain.invoke(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

In [267]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 1923
Prompt Tokens: 1438
Completion Tokens: 485
Total Cost: 0.0014465


In [269]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [272]:
quiz=response.get('quiz')

In [276]:
quiz = json.loads(quiz)

In [277]:
quiz_table_data=[]
for key, value in quiz.items():
    mcq = value['mcq']
    options = " | ".join(
        [
            f"{option}:{option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value['correct']
    quiz_table_data.append({'MCQ':mcq,"Choices":options, "Correct":correct})

In [278]:
quiz_table_data

[{'MCQ': 'Who coined the term machine learning in 1959?',
  'Choices': 'a:Donald Hebb | b:Tom M. Mitchell | c:Arthur Samuel | d:Warren McCulloch',
  'Correct': 'c'},
 {'MCQ': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'Choices': 'a:Cyberdyne | b:Cybertron | c:Robotech | d:Skynet',
  'Correct': 'b'},
 {'MCQ': 'Who proposed the early mathematical models of neural networks to come up with algorithms that mirror human thought processes?',
  'Choices': 'a:Walter Pitts | b:Tom M. Mitchell | c:Donald Hebb | d:Arthur Samuel',
  'Correct': 'a'},
 {'MCQ': 'What is the objective of modern-day machine learning related to classifying data?',
  'Choices': 'a:To predict future outcomes | b:To make decisions for the user | c:To classify data based on developed models | d:To analyze historical data',
  'Correct': 'c'},
 {'MCQ': 'According to Tom M. Mitchell, when is a computer program said to learn from experience?',
  'Choices': 'a:W

In [280]:
quiz = pd.DataFrame(quiz_table_data)

In [281]:
quiz.to_csv("machinelearning.csv",index=False)